In [1]:
import sys
sys.path.append('..')

from classes.NetworkXGraph import Graph

In [2]:
# Building graph

    # source: https://graphchallenge.mit.edu/data-sets
    # file_source: https://graphchallenge.s3.amazonaws.com/snap/as20000102/as20000102_adj.tsv
    # file_format: Adjacency TSV (Tab-Separated Values)
        # <dataset-name>_adj.tsv
        # (Row, Col, Value) tuple describing the adjacency matrix of the graph in tab separated format.
        # Adjacency matrix is of size Num_vertices x Num_vertices
    # file: as20000102_adj.tsv (Autonomous Systems graph from January 02 2000)
    # num of nodes:      6.474
    # num of edges:     25.144
    # num of triangles:  6.584

g = Graph()
g.build_from_tsv('../datasets/as20000102_adj.tsv', data=(('ajd_value',float),), show_info=['bandwidth'])

Number of lines read from TSV file: 25144
Name: 
Type: Graph
Number of nodes: 6474
Number of edges: 12572
Average degree:   3.8838
Connected:  Yes
Bandwidth:  12943
Reduced Bandwidth:  8727
elapsed time in seconds: 1.536594


In [3]:
# Calcular a redução de banda do grafo original.
original_bandwidth = g._means['bandwidth']
original_reduced_bandwidth = g._means['reduced_bandwidth']
graph = g._graph

len(graph.nodes)

6474

In [4]:
import numpy as np
from queue import Queue

import networkx as nx
from networkx.utils import reverse_cuthill_mckee_ordering

def get_bandwidth(G):
    L = nx.laplacian_matrix(G, nodelist=sorted(G.nodes())) # Laplacian Matrix
    x,y = np.nonzero(L)
    return (y-x).max()+(x-y).max()+1

def print_nodes_info(G):
    print("------------------------------------------")
    for n in sorted(G.nodes()):
        print('Node %d -> id: %s / degree: %d' % (n, G.nodes[n]['id'], G.degree(n)) )
    print("------------------------------------------")
    
def get_sample_nodes_list(G):
    sample = list()
    q = Queue()
    max_len = int(len(graph.nodes) * 0.1) # amostra com tamanho de 10 por cento do original
    #max_len = 8

    initial_node = np.random.choice(graph.nodes, 1)[0]
    #initial_node = 2338
    q.put(initial_node)

    while len(sample) < max_len:
        node = q.get()
        sample.append(node)
        neighbors = [n for n in graph.neighbors(node) if n not in sample and n not in q.queue]
        for n in neighbors:
            q.put(n)
    return sample


In [5]:
reduced_bandwidth = list()

for i in range(10):
    #1) Obter amostras do grafo original: a partir de um nó aleatório n1, realizar uma busca em grafo até uma quantidade K
    sample = get_sample_nodes_list(graph)
    #print('sample with %d nodes' % len(sample))

    #2) Aplicar a redução de banda nessa amostra e obter os novos rótulos dos nós.
    sample_graph = graph.subgraph(sample).copy()
    #print('Bandwidth of original sample: %d' % get_bandwidth(sample_graph))
    rcm = list(reverse_cuthill_mckee_ordering(sample_graph))
    mapping = dict(zip(rcm, sample_graph.nodes))
    #sample_graph = nx.relabel_nodes(sample_graph, mapping)
    #print('Reduced bandwidth of sample: %d' % get_bandwidth(sample_graph))

    #3) Aplicar no grafo original os novos rótulos obtidos
    #4) Recalcular a redução de banda do grafo original.
    print('-------------------------------------------------------')
    print('Bandwidth of graph: %d' % get_bandwidth(graph))
    graph = nx.relabel_nodes(graph, mapping)
    reduced_bandwidth.append(get_bandwidth(graph))
    print('Reduced bandwidth of graph: %d' % reduced_bandwidth[i])
    

-------------------------------------------------------
Bandwidth of graph: 12943
Reduced bandwidth of graph: 12783
-------------------------------------------------------
Bandwidth of graph: 12783
Reduced bandwidth of graph: 12719
-------------------------------------------------------
Bandwidth of graph: 12719
Reduced bandwidth of graph: 12815
-------------------------------------------------------
Bandwidth of graph: 12815
Reduced bandwidth of graph: 12707
-------------------------------------------------------
Bandwidth of graph: 12707
Reduced bandwidth of graph: 12725
-------------------------------------------------------
Bandwidth of graph: 12725
Reduced bandwidth of graph: 12707
-------------------------------------------------------
Bandwidth of graph: 12707
Reduced bandwidth of graph: 12707
-------------------------------------------------------
Bandwidth of graph: 12707
Reduced bandwidth of graph: 12707
-------------------------------------------------------
Bandwidth of gra

In [6]:
#5) Obter um novo nó (distante de n1) e repetir o processo.
#6) Ao final de x passos, plotar a redução de banda obtida em cada passo em relação a redução original.

In [7]:
# Using samples to build graphs and to calculate

#g.get_analisys_from_samples(['reduced_bandwidth'])

In [8]:
# Ploting results

#g.plot_analisys(['reduced_bandwidth'])